# Modeling the Covid-19 Death rate relative to population size
In this step we are finding the right or best model to correlate death rates with the number of cases and the demography of a country. This will help us understand whether we can tell how harmful COVID 19 is for a given country based on its demography. Our model could be tested with more recent data of countries that did not have had confirmed cases of COVID-19 back in May.

In [1]:
#load python packages
import os
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score,mean_absolute_error
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
%matplotlib inline
os.getcwd()

/Users/lisahw/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/lisahw/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


'/Users/lisahw/Documents/Courses and Conferences/DataScience/MyProject/Capstone_02/Springboard/notebooks'

### Read in Train and Test data sets

In [183]:
data_file = 'sqlite:///../data/interim/COVID_Deaths_train_test.db'
X_train = pd.read_sql('SELECT * FROM XTRAIN',data_file,index_col='index')
X_test = pd.read_sql('SELECT * FROM XTEST',data_file,index_col='index')
y_train = np.ravel(pd.read_sql('SELECT * FROM yTRAIN',data_file,index_col='index'))
y_test = np.ravel(pd.read_sql('SELECT * FROM yTEST',data_file,index_col='index'))
X_train.head()

,Confirmed,Cardio Death Rate,Diabetes Percentage,Obesity,Undernourished,PopMale,Total Population,covid_fatal,Cluster_1,Cluster_2
index,,,,,,,,,,
145,-0.612502,-0.188623,-0.897430,-1.282407,2.990430,-1.073046,-0.191597,-0.344486,0,0
42,0.795560,-0.977732,-0.467855,0.081550,-0.228392,-0.211033,-0.194520,0.357085,0,1
16,-0.552803,0.344180,0.287237,-1.612741,0.335731,-0.448477,0.679257,-0.618901,0,0
10,-0.514918,0.179408,-0.153011,0.859433,-0.560230,-0.321158,-0.040489,1.025781,1,0
115,-0.494024,-0.036122,1.151723,0.561067,-0.410903,0.294547,-0.298068,-0.967417,1,0


Data has been standardized and clusters identified. There are 3 clusters, but one has been dropped to avoid covariance.

## Linear Regression Model

In [184]:
from sklearn.linear_model import LinearRegression

linreg = LinearRegression()
linreg.fit(X_train,y_train)
y_pred = linreg.predict(X_test)
print('Explained Variance: ',explained_variance_score(y_test,y_pred))
print('Mean square error: ',mean_squared_error(y_test,y_pred))
print('Mean absolute error: ',mean_absolute_error(y_test,y_pred))
print('R2 score/Coeff. of determination: ',linreg.score(X_test,y_test))

Explained Variance:  0.6709053022383749
Mean square error:  0.9024420689143245
Mean absolute error:  0.4504206706889954
R2 score/Coeff. of determination:  0.6474097594592294


The linear model does not perform very well on our data set. Potentially the underlying data is better described by a polynomial function with a degree higher than 1.

In [153]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
polyfeat = PolynomialFeatures(degree=2,include_bias=False)
linreg = LinearRegression()
pipeline = Pipeline([("polynomial_features", polyfeat),("linear_regression", linreg)])
pipeline.fit(X_train,y_train)
y_pred = pipeline.predict(X_test)
print('Explained Variance: ',explained_variance_score(y_test,y_pred))
print('Mean square error: ',mean_squared_error(y_test,y_pred))
print('Mean absolute error: ',mean_absolute_error(y_test,y_pred))
print('R2 score/Coeff. of determination: ',pipeline.score(X_test,y_test))

Explained Variance:  0.7884672004546828
Mean square error:  0.6030959339957563
Mean absolute error:  0.3936775373579578
R2 score/Coeff. of determination:  0.7643663258157434


The fit to the linear model is highly improved thanks to adding the polynomial features.

## Decision Tree Regressor

Decision trees are usually used for classification problems, but are of value for a regression problem too. 

In [158]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg.fit(X_train, y_train)
y_pred = tree_reg.predict(X_test)

print('Mean square error: ',mean_squared_error(y_test,y_pred))
print('R2 score/Coeff. of determination: ',r2_score(y_test,y_pred))

Mean square error:  1.0038786953973788
R2 score/Coeff. of determination:  0.6077777811159194


In [134]:
param_grid = {"ccp_alpha": [0.0,0.5],
 "max_depth": [2,3, None],
 "max_features": [3, 6, 9],
 "min_samples_split": [2, 5],
 "min_samples_leaf": [1, 3]}
reg3 = DecisionTreeRegressor(random_state=42)
grid = GridSearchCV(estimator=reg3, param_grid=param_grid, n_jobs=-1,cv=2)
grid.fit(X_train, y_train)
print('Best score: ',grid.best_score_)
print('Best set of parameters: ' ,grid.best_params_)
y_pred = grid.predict(X_test)
print('Mean square error: ',mean_squared_error(y_test,y_pred))
print('R2 score/Coeff. of determination: ',r2_score(y_test,y_pred))

Best score:  0.5176525447980902
Best set of parameters:  {'ccp_alpha': 0.0, 'max_depth': 3, 'max_features': 6, 'min_samples_leaf': 3, 'min_samples_split': 2}
Mean square error:  1.0010994002666564
R2 score/Coeff. of determination:  0.608863670584541


The crossvalidation has slightly improved the model performance. The polynomial model still outperforms the Decision Tree Regressor.

In [140]:
tree_reg = DecisionTreeRegressor(max_depth=3, random_state=42, max_features=6,min_samples_leaf=3)
tree_reg.fit(X_train, y_train)
y_pred = tree_reg.predict(X_test)

## Random Forest Regressor

In [185]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(random_state=0, n_estimators=250)
# regressor = RandomForestRegressor(bootstrap= False,max_depth= 3, max_features= 6, min_samples_leaf= 3, min_samples_split=2,n_estimators= 250)
regressor.fit(X_train.values, y_train)
y_pred = regressor.predict(X_test.values)
print('Mean square error: ',mean_squared_error(y_test,y_pred))
print('R2 score/Coeff. of determination: ',r2_score(y_test,y_pred))

Mean square error:  0.5628122812402983
R2 score/Coeff. of determination:  0.780105422323063


In [186]:
param_grid = {"n_estimators": [150,250],
 "max_depth": [3, None],
 "max_features": [3, 6, 9],
 "min_samples_split": [2, 5],
 "min_samples_leaf": [1, 3],
 "bootstrap": [True, False]}
reg2 = RandomForestRegressor(random_state=0)
grid = GridSearchCV(estimator=reg2, param_grid=param_grid, n_jobs=-1,cv=2)
grid.fit(X_train.values, y_train)
print('Best score: ',grid.best_score_)
print('Best set of parameters: ' ,grid.best_params_)
y_pred = grid.predict(X_test.values)
print('Mean square error: ',mean_squared_error(y_test,y_pred))
print('R2 score/Coeff. of determination: ',r2_score(y_test,y_pred))

Best score:  0.6930610713723051
Best set of parameters:  {'bootstrap': True, 'max_depth': None, 'max_features': 3, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 150}
Mean square error:  0.7791700934062301
R2 score/Coeff. of determination:  0.6955729568472064


Instead of just one decision tree, a Random Forest regressor is now used to model the data set. R2 score shows that the Random Forest regressor outperforms the single Decision Tree, but it does not compete with the polynomial Model.

A benefit of the Random Forest is that it provides the relative relevance of each feature. 

In [187]:
regressor = RandomForestRegressor(random_state=0, n_estimators=250)
regressor.fit(X_train.values, y_train)
print('Feature Importance:')
for col,imp in zip(X_train.columns,regressor.feature_importances_):
    print('{} : {:.1f}%'.format(col,100*imp))

Feature Importance:
Confirmed : 71.0%
Cardio Death Rate : 1.7%
Diabetes Percentage : 4.0%
Obesity : 2.1%
Undernourished : 0.3%
PopMale : 4.2%
Total Population : 1.1%
covid_fatal : 15.5%
Cluster_1 : 0.1%
Cluster_2 : 0.1%


We see that the most important feature is the number of comfirmed cases. This was to be expected, since deaths are correlated to incidents. Interestingly, the Diabetes Percentage appears to be more relevant than the fraction of male population above 70. But with our mission to predict which other countries might be affected but do not have confirmed cases yet, this result does not help us.

## Gradient Boosting Regressor

In [188]:
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor(random_state=42)
# Fit the model on the trainng data.
gbr.fit(X_train, y_train)
# Print the accuracy from the testing data.
y_pred = gbr.predict(X_test)
print('Explained Variance: ',explained_variance_score(y_test,y_pred))
print('Mean square error: ',mean_squared_error(y_test,y_pred))
print('Mean absolute error: ',mean_absolute_error(y_test,y_pred))
print('R2 score/Coeff. of determination: ',r2_score(y_test,y_pred))

Explained Variance:  0.856775970292404
Mean square error:  0.39550673735017344
Mean absolute error:  0.22919703239615866
R2 score/Coeff. of determination:  0.8454728337016033


In [174]:
gbr.get_params

<bound method BaseEstimator.get_params of GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=42, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)>

In [190]:
param_grid = {"n_estimators": [100,150,250],
              "subsample": [0.5,0.75,1],
 "max_depth": [3, None],
 "max_features": [3, 6, 9],
 "min_samples_split": [2, 5],
 "min_samples_leaf": [1, 3]}
gbr = GradientBoostingRegressor(random_state=42)
grid = GridSearchCV(estimator=gbr, param_grid=param_grid, n_jobs=-1,cv=2)
grid.fit(X_train, y_train)
print('Best score: ',grid.best_score_)
print('Best set of parameters: ' ,grid.best_params_)
y_pred = grid.predict(X_test)
print('Mean square error: ',mean_squared_error(y_test,y_pred))
print('R2 score/Coeff. of determination: ',r2_score(y_test,y_pred))

Best score:  0.7603010304765294
Best set of parameters:  {'max_depth': 3, 'max_features': 9, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 250, 'subsample': 0.5}
Mean square error:  0.5108937747810689
R2 score/Coeff. of determination:  0.8003903351297101


In [189]:
gbr = GradientBoostingRegressor(random_state=42,max_depth=3,max_features=3,n_estimators=150)
gbr.fit(X_train,y_train)
for col,imp in zip(X_train.columns,gbr.feature_importances_):
    print('{} : {:.1f}%'.format(col,100*imp))

Confirmed : 25.5%
Cardio Death Rate : 3.4%
Diabetes Percentage : 7.4%
Obesity : 3.2%
Undernourished : 4.7%
PopMale : 21.0%
Total Population : 4.0%
covid_fatal : 25.1%
Cluster_1 : 0.0%
Cluster_2 : 5.6%


The Gradient Boosting model outperforms the Random Forest, but does not perform better than the polynominal model. Interestingly, the Gradient Boosting gives less importance to the number of confirmed cases and increases the importance of the percentage of males above 70. The diabetes percentage is on rank three this time.

In [179]:
y_train.values.reshape((y_train.shape[0])).shape

(110,)

In [182]:
np.ravel(y_train).shape

(110,)